In [2]:
import scanpy as sc
import numpy as np
orig_path="/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/adata_Training.h5ad"
data_path="/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/arc/perturb_processed.h5ad"
adata = sc.read_h5ad(orig_path)
adata.obs["condition"] = adata.obs["target_gene"]
adata.obs["condition"] = (
    adata.obs["target_gene"].replace("non-targeting","ctrl")
)
adata.obs["condition"]=adata.obs['condition'].cat.rename_categories({cat:f"ctrl+{cat}" for cat in adata.obs.condition.cat.categories if cat != "ctrl"})
adata.var["gene_name"] = adata.var.index.values
adata.obs["cell_type"] = "h1_ESC"
adata.obs["dose_val"] = np.where(adata.obs["condition"] == "ctrl", "1", "1+1")
adata.obs["condition_name"] = adata.obs["cell_type"].astype(str)+"_"+ adata.obs["condition"].astype(str) + "_" + adata.obs["dose_val"].astype(str) 
adata.write_h5ad(filename=data_path)
print(adata)
print(adata.obs)
del adata

: 

In [2]:
import scanpy as sc
data_path="/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/arc/perturb_processed.h5ad"
adata = sc.read_h5ad(data_path)
import pickle as pk
with open("/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/gene2go_all.pkl","rb") as f:
    datago = pk.load(f)

In [3]:

key_set = set(datago.keys())
genes = set(adata.var_names)         # O(n)
common  = genes & key_set            # O(min(n, m))
count   = len(common)

print(count)


17525


In [5]:
import scanpy as sc 
test = sc.read_h5ad("/scratch/saigum/PerturbationPredictionUsingGeneNetworks/demo/data/norman/perturb_processed.h5ad")
print(test.obs)

                        condition cell_type dose_val  control  \
cell_barcode                                                    
AAACCTGAGGCATGTG-1   TSC22D1+ctrl      A549      1+1        0   
AAACCTGAGGCCCTTG-1    KLF1+MAP2K6      A549      1+1        0   
AAACCTGCACGAAGCA-1           ctrl      A549        1        1   
AAACCTGCAGACGTAG-1  CEBPE+RUNX1T1      A549      1+1        0   
AAACCTGCAGCCTTGG-1     MAML2+ctrl      A549      1+1        0   
...                           ...       ...      ...      ...   
TTTGTCAGTCATGCAT-8   RHOXF2BB+SET      A549      1+1        0   
TTTGTCATCAGTACGT-8     FOXA3+ctrl      A549      1+1        0   
TTTGTCATCCACTCCA-8     CELF2+ctrl      A549      1+1        0   
TTTGTCATCCCAACGG-8    BCORL1+ctrl      A549      1+1        0   
TTTGTCATCTGGCGAC-8    MAP4K3+ctrl      A549      1+1        0   

                            condition_name  
cell_barcode                                
AAACCTGAGGCATGTG-1   A549_TSC22D1+ctrl_1+1  
AAACCTGAGGCCCTTG-1 

In [7]:
test.obs["condition_name_2"] = test.obs["cell_type"].astype(str)+"_"+ test.obs["condition"].astype(str) + "_" + test.obs["dose_val"].astype(str) 


In [11]:
(test.obs["condition_name_2"] == test.obs["condition_name"]).sum()

np.int64(91205)

In [2]:
test.var

,gene_name
gene_id,
ENSG00000239945,RP11-34P13.8
ENSG00000223764,RP11-54O7.3
ENSG00000187634,SAMD11
ENSG00000187642,PERM1
ENSG00000188290,HES4
...,...
ENSG00000198786,MT-ND5
ENSG00000198695,MT-ND6
ENSG00000198727,MT-CYB


In [1]:
from gears import PertData
pert_data = PertData('./vcc_data') # specific saved folder
pert_data.load(data_path="/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/arc/")


/scratch/saigum/PerturbationPredictionUsingGeneNetworks/perturb/lib/python3.13/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
Found local copy...


OSError: Unable to synchronously open file (bad object header version number)

In [ ]:
pert_data.load(data_path="/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/arc/")


In [ ]:
pert_data.prepare_split(split = 'simulation', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader